# Defining all libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -v 'nibabel==3.2.1'
!pip install tensorflow_addons
import os
import random
import re
import scipy
import scipy.misc
import numpy as np
from skimage import exposure
import cv2
import nibabel as nib
from nilearn.image import resample_img
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import rcParams
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
import keras.layers as tfl
import keras.models as models
import tensorflow.python.keras.backend as K
from zipfile import ZipFile
plt.set_cmap('gray')
%matplotlib inline

# Image directory and size parameters

In [ ]:
HOME_DIR = os.path.join("/content/drive/MyDrive/PROJECT/Segmentation/")
MODEL_DIR = os.path.join(HOME_DIR,"Models")
LOG_DIR = os.path.join(HOME_DIR,"Logs")
PLOT_DIR = os.path.join(HOME_DIR,"Plots")
IMAGE_DIR = os.path.join(HOME_DIR,"Images")
DATA_DIR = os.path.join(HOME_DIR,"Dataset")
NII_DIR = os.path.join(DATA_DIR, "CT-ORG")

TRAIN_DIR = os.path.join(NII_DIR, "train")
TEST_DIR = os.path.join(NII_DIR, "test")

MASKS_NII = "MASKS_NII/"
PATIENT_NII = "PATIENT_NII/"
TRAIN = "train/"
TEST = "test/"

In [ ]:
IMG_DTYPE = float
MASK_DTYPE = np.uint8
image_size = 256
batch_size = 8
epochs = 10
IMG_DIM=(image_size,image_size,300)

## Seeding
seed = 2019
random.seed = seed
np.random.seed = seed
tf.seed = seed

In [ ]:
directory_to_move =  PATIENT_NII
desired_scans_path = os.path.join(TRAIN_DIR, PATIENT_NII)
desired_masks_path = os.path.join(TRAIN_DIR, MASKS_NII)

test_scans_path = os.path.join(TEST_DIR, PATIENT_NII)
test_masks_path = os.path.join(TEST_DIR, MASKS_NII)

# Utility functions

In [ ]:
def getIDFromName(array):
    numbers = []
    for string in array:
        match = re.search(r'\d+', string)
        if match:
            number = int(match.group())
            numbers.append(number)
    return numbers


In [ ]:
def show_nifti_slices(slices):
  """ Function to display row of image slices """
  fig, axes = plt.subplots(1, len(slices))
  for i, slice in enumerate(slices):
      axes[i].imshow(slice.T, cmap="gray", origin="lower")

In [ ]:
## Training Ids

train_list = []
for file in os.listdir(os.path.join(desired_scans_path)):
        train_list.append(file)

print(len(train_list))

val_data_size = len(train_list)//5

valid_ids = getIDFromName(train_list[:val_data_size])
train_ids = getIDFromName(train_list[val_data_size:])

print(train_ids)

In [ ]:
CLASSES = ['background', 'liver', 'bladder','lung', 'kidneys', 'bones' , 'brain']
num_classes = len(CLASSES)

labels = {}
for i in range(len(CLASSES)):
    labels[i] = CLASSES[i]

print(labels)


# Pre-processing Methods

## Histogram Equalization

In [ ]:
def equalize_histogram(image):
    image_equalized = exposure.equalize_hist(image)
    return image_equalized

# Example usage
sample_image_path = os.path.join(TRAIN_DIR,PATIENT_NII, "volume-23.nii.gz")
sample_image = nib.load(sample_image_path).get_fdata()
equalized_image = equalize_histogram(sample_image)
plt.imshow(equalized_image[:,:,50], cmap='gray')
plt.title("Equalized Image")
plt.show()

## Normalize

In [ ]:
def normalize_image(img):
    """ Normalize image values to [0,1] """
    norm = cv2.normalize(img, None, alpha=0.0, beta=1.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    return norm

## Resizing

In [ ]:
from skimage.transform import resize

def resize_image(image, target_shape=(128, 128, 128)):
    resized_image = resize(image, target_shape, mode='constant')
    return resized_image

# Example usage
sample_image_path = os.path.join(TRAIN_DIR, PATIENT_NII, "volume-23.nii.gz")
sample_image = nib.load(sample_image_path).get_fdata()
resized_image = resize_image(sample_image)
plt.imshow(resized_image[:,:,50], cmap='gray')
plt.title("Resized Image")
plt.show()
